# **Reciprocal Space and Brillouin Zone**

**Authors:** Taylor James Baird, Dou Du and Giovanni Pizzi

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/master/notebook/band-theory/brillouin_zone.ipynb
<p style="text-align: justify;font-size:15px">
    In this notebook, we explore two central concepts in electronic structure and band theory: those of reciprocal space and the Brillouin zone.
</p>
<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**
* Appreciate the nature of the relationship between the real and reciprocal space descriptions of a crystalline system.
* Use this understanding to explain the differences in the Brillouin zones observed for different crystal structures. 


## **Background theory** 

[More on the background theory.](./theory/theory_brillouin_zone.ipynb)

## **Tasks and exercises** 

<ol style="text-align: justify;font-size:15px">
    <li> What is the relationship between real space primitive vectors and reciprocal space
        primitive vectors? 
    <details>
    <summary style="color: red">Solution</summary>
    The definition of the reciprocal lattice vector $\vec{b}_1$ is:
    $$\vec{b}_1 = 2\pi \frac{\vec{a}_2 \times \vec{a}_3}{\vec{a}_1 \cdot (\vec{a}_2 \times \vec{a}_3)}$$
    Hence, the dot products of each real space primitive vectors with $\vec{b}_1$ are
    computed as:
    $$\vec{a}_1 \cdot \vec{b}_1 = 2 \pi \vec{a}_1  \cdot  \frac{\vec{a}_2 \times \vec{a}_3}{\vec{a}_1 \cdot (\vec{a}_2 \times \vec{a}_3)} = 2  \pi$$
    $$\vec{a}_2 \cdot  \vec{b}_1 =2 \pi  \vec{a}_2 \cdot  \frac{\vec{a}_2 \times \vec{a}_3}{\vec{a}_1 \cdot (\vec{a}_2 \times \vec{a}_3)} = 0$$
    $$\vec{a}_3 \cdot  \vec{b}_1 =2 \pi  \vec{a}_3 \cdot  \frac{\vec{a}_2 \times \vec{a}_3}{\vec{a}_1 \cdot (\vec{a}_2 \times \vec{a}_3)} = 0$$
    Similarly, one can compute for other reciprocal lattice vector $\vec{b}_2$ and 
    $\vec{b}_3$. In summary, we have the relation as $\vec{a}_i \cdot \vec{b}_j = 2\pi \delta _{ij}$.
    </details>
    </li>
    <li> What is the volume of the primitive cell in reciprocal space? How it is related 
         to the volume of the real space primitive cell? Please give a mathematical proof.  
    <details>
    <summary style="color: red">Solution</summary>
    The volume of the primitive cell in reciprocal space, $\Omega_b$, can be calculated as:
        $$\Omega_b =  \vec{b}_1 \cdot (\vec{b}_2 \times \vec{b}_3) = 
        2\pi \frac{\vec{a}_2 \times \vec{a}_3}{\vec{a}_1 \cdot (\vec{a}_2 \times \vec{a}_3)}
        (\vec{b}_2 \times \vec{b}_3)$$
    According to the Lagrange's identity, one can show:
        $$(\vec{a}_2 \times \vec{a}_3) \cdot (\vec{b}_2 \times \vec{b}_3) =
        (\vec{a}_2 \cdot \vec{b}_2)(\vec{a}_3 \cdot \vec{b}_3) - 
        (\vec{a}_2 \cdot \vec{b}_3)(\vec{a}_3 \cdot \vec{b}_2) = 2\pi \cdot 2\pi - 0 = 4\pi^2$$
    Hence,
        $$\Omega_b = \frac{8\pi^3}{\vec{a}_1 \cdot (\vec{a}_2 \times \vec{a}_3)} = \frac{8\pi^3}{\Omega_a}$$
    where, $\Omega_a$ is the volume of the real space primitive cell.
    </details>   
    </li>
    <li> What are the structures of the Wigner-Seitz primitive cells for the BCC and FCC lattices? How about their 1st Brillouin zones?
    <details>
    <summary style="color: red">Solution</summary>
    <div style="text-align:center">
    <img src="./images/BCC.png" alt="drawing" style="width:300px;"/>
    <figcaption>Figure 1. Demonstration of the procedure for constructing the primitive lattice vectors for the body-centered cubic (BCC) structure.</figcaption>
    </div>
    For a BCC structure, as shown in the Figure 1, the primitive lattice vectors can
    be constructed as:
        $$\vec{a}_1 = \frac{a}{2} (\vec{x}+\vec{y}-\vec{z})$$
        $$\vec{a}_2 = \frac{a}{2} (-\vec{x}+\vec{y}+\vec{z})$$
        $$\vec{a}_3 = \frac{a}{2} (\vec{x}-\vec{y}+\vec{z})$$
        
    The corresponding reciprocal lattice vectors are:
        $$\vec{b}_1 = 2\pi \frac{\vec{a}_2 \times \vec{a}_3}
        {\vec{a}_1 (\vec{a}_2 \times \vec{a}_3)} = \frac{2\pi}{\Omega}
        (\vec{a}_2 \times \vec{a}_3) = \frac{\pi a^2}{\Omega} (x + y)$$
        
        $$\vec{b}_2 = 2\pi \frac{\vec{a}_3 \times \vec{a}_1}
        {\vec{a}_1 (\vec{a}_2 \times \vec{a}_3)} = \frac{2\pi}{\Omega}
        (\vec{a}_3 \times \vec{a}_1) = \frac{\pi a^2}{\Omega} (y + z)$$
        
        $$\vec{b}_1 = 2\pi \frac{\vec{a}_1 \times \vec{a}_2}
        {\vec{a}_1 (\vec{a}_2 \times \vec{a}_3)} = \frac{2\pi}{\Omega}
        (\vec{a}_2 \times \vec{a}_3) = \frac{\pi a^2}{\Omega} (x + y)$$
        
    <div style="text-align:center">
    <img src="./images/FCC.png" alt="drawing" style="width:300px;"/>
    <figcaption>Figure 2.Demonstration of the procedure for constructing the primitive lattice vectors for the face-centered cubic (FCC) structure.</figcaption>
    </div>
        
    For an FCC structure as shown in Figure 2, the primitive lattice vectors can
    be constructed as:
        $$\vec{a}_1 = \frac{a}{2} (\vec{x}+\vec{y})$$
        $$\vec{a}_2 = \frac{a}{2} (\vec{y}+\vec{z})$$
        $$\vec{a}_3 = \frac{a}{2} (\vec{z}+\vec{x})$$
    
    The corresponding reciprocal lattice vectors are:
    
    $$\vec{b}_1 = 2\pi \frac{\vec{a}_2 \times \vec{a}_3}
        {\vec{a}_1 \cdot (\vec{a}_2 \times \vec{a}_3)} = \frac{2\pi}{\Omega}
        (\vec{a}_2 \times \vec{a}_3) = \frac{\pi a^2}{2 \Omega}
        (\vec{x}+\vec{y}-\vec{z})$$
        
    $$\vec{b}_2 = 2\pi \frac{\vec{a}_3 \times \vec{a}_1}
        {\vec{a}_1 \cdot (\vec{a}_2 \times \vec{a}_3)} = \frac{2\pi}{\Omega}
        (\vec{a}_3 \times \vec{a}_1) = \frac{\pi a^2}{2 \Omega}
        (-\vec{x}+\vec{y}+\vec{z})$$
        
    $$\vec{b}_1 = 2\pi \frac{\vec{a}_1 \times \vec{a}_2}
        {\vec{a}_1 \cdot (\vec{a}_2 \times \vec{a}_3)} = \frac{2\pi}{\Omega}
        (\vec{a}_2 \times \vec{a}_3) = \frac{\pi a^2}{2 \Omega}
        (\vec{x}-\vec{y}+\vec{z})$$
    
    One can see that the reciprocal lattice vectors of the BCC structure are simply 
    the real space primitive lattice vectors of the FCC structure. Conversely, the reciprocal lattice of the FCC structure
    is just that of BCC structure in real space. In other words, the 1st Brillouin zone of the BCC structure
    has the same structure as the Wigner-Seitz primitive cell of FCC.
    And the 1st Brillouin zone of the FCC lattice has the same structure as the
    the Wigner-Seitz primitive cell of BCC.
    </details>   
    </li>
</ol>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [ ]:
from widget_bzvisualizer import BZVisualizer
import numpy as np
import seekpath
import nglview as nv
from ase.build import bulk, molecule
from ipywidgets import HBox, VBox, Button, Output, Text, Tab, Layout, Label, HTML, Dropdown
import ipywidgets as widgets
import nglview as nv
from ase import Atom, Atoms
from ase.symbols import symbols2numbers
from ase.data import atomic_numbers
from ase.lattice.cubic import BodyCenteredCubic,SimpleCubic, FaceCenteredCubic
from ase.lattice.orthorhombic import *
from ase.lattice.triclinic import Triclinic
from ase.lattice.monoclinic import *
from ase.lattice.tetragonal import *
import logging
from ase.lattice.hexagonal import Hexagonal

In [ ]:

mat = SimpleOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': 3.6, 'b': 3.6, 'c': 3.6})


m = nv.NGLWidget(width='400px', height='400px')
m.background='black'
c1 = m.add_component(nv.ASEStructure(mat))

m.clear()
m.add_spacefill(radius=0.4)
m.add_unitcell()
# m.add_axes()
mpos = [];
msym = [];

for i in mat:
    mpos.append(i.position.tolist())
    msym.append('Cu')

a=3.6
b=3.6
c=3.6
real_lattice=np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) *a/ 2.0;

w = BZVisualizer(real_lattice, [[0.0, 0.0, 0.0]], [1], True, height='400px',width='400px', enable_interaction=True)

# dictionary to keep track of lattice parameters for easy reversion and comparison
param_dict = {'Cubic': {'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90,'gamma':90},\
'Cubic-Primitive': {'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90,'gamma':90},\
'Cubic-Body-centered': {'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90,'gamma':90},\
'Cubic-Face-centered': {'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90,'gamma':90},\
'Hexagonal-Primitive':{'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90, 'gamma':90},\
'Rhombohedral-Primitive':{'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90, 'gamma':90},\
'Tetragonal-Primitive':{'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90, 'gamma':90},\
'Tetragonal-Body-centered':{'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90, 'gamma':90},\
'Orthorhombic-Primitive':{'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90, 'gamma':90},\
'Orthorhombic-Face-centered':{'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90, 'gamma':90},\
'Orthorhombic-Body-centered':{'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90, 'gamma':90},\
'Orthorhombic-Base-centered':{'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90, 'gamma':90},\
'Monoclinic-Primitive':{'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90, 'gamma':90},\
'Monoclinic-Base-centered':{'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90, 'gamma':90},\
'Triclinic':{'a':3.0,'b':3.0,'c':3.0,'alpha':90,'beta':90, 'gamma':90}}


In [ ]:
"""
now contents are dropdwn menus. 
Depending on the crystal family selected, there shall be a dropdown menu 

"""
crystal_family = 'Cubic'
crystal_family_radio = widgets.RadioButtons(options=['Cubic','Hexagonal','Rhombohedral','Tetragonal','Orthorhombic','Monoclinic','Triclinic'],
                                           value='Cubic',    
                                           disabled=False)

centering_radio = widgets.RadioButtons(options=['Primitive','Body-centered', 'Face-centered'],
                                           value='Primitive',
                                           disabled=False)


crystal_family_title=Text(value='Crystal family')
col0 = VBox([crystal_family_title, crystal_family_radio])
centering_title=Text(value='Centering')
col1 = VBox([centering_title, centering_radio])


# Callback which reverts lattice parameters to their saved values.
def on_revert_lattice_params(change):
    
    crystal_family = crystal_family_radio.value
    centering=centering_radio.value
    a_slider.value=param_dict[crystal_family+'-'+centering]['a']
    b_slider.value=param_dict[crystal_family+'-'+centering]['b']
    c_slider.value=param_dict[crystal_family+'-'+centering]['c']
    alpha_slider.values=param_dict[crystal_family+'-'+centering]['alpha'] 
    beta_slider.values=param_dict[crystal_family+'-'+centering]['beta'] 
    gamma_slider.values=param_dict[crystal_family+'-'+centering]['gamma']
    
    compute_BZ()
    
def on_save_lattice_params(change):
    
    crystal_family = crystal_family_radio.value

    
    a = a_slider.value
    b = b_slider.value
    c = c_slider.value

    alpha=alpha_slider.value
    beta=beta_slider.value
    gamma=gamma_slider.value
    

    centering=centering_radio.value
    
    # save lattice parameters for retrieval after a change of crystal family
    
  
    param_dict[crystal_family+'-'+centering]['a'] = a
    param_dict[crystal_family+'-'+centering]['b'] = b
    param_dict[crystal_family+'-'+centering]['c'] = c
    param_dict[crystal_family+'-'+centering]['alpha'] = alpha
    param_dict[crystal_family+'-'+centering]['beta'] = beta
    param_dict[crystal_family+'-'+centering]['gamma'] = gamma
    


def on_orientation_sync(c):
    ngl_dict=m.__dict__['_trait_values']
    init_ngl_matrix= np.array([24.451005287902106, 0.0, 0.0, -3.5999999046325684, 0.0, 24.451005287902106, 0.0, -3.5999999046325684, 0.0, 0.0, 24.451005287902106, -3.5999999046325684, 0.0, 0.0, 0.0, 1.0]).reshape((4,4))
    temp = (np.linalg.inv(init_ngl_matrix))@np.array(ngl_dict["_camera_orientation"]).reshape(4,4).T
    temp2 = (np.array(ngl_dict["_camera_orientation"]).reshape(4,4).T)@(np.linalg.inv(init_ngl_matrix))
    ngl_extmat=np.array(ngl_dict["_camera_orientation"]).reshape(4,4).T
    init_worldmatrix = np.array([1,0,0,0,0,1,0,0,0,0,1,0,0,0,10.471975511965978,1]).reshape((4,4)).T
    ngl_extmat = init_worldmatrix@temp
#     orientation = init_worldmatrix@temp
#     orientation = temp@init_worldmatrix
#     ngl_extmat = temp@np.linalg.inv(init_worldmatrix)
#     ngl_extmat = temp@init_worldmatrix

#     ngl_extmat = temp2@init_worldmatrix

    R=ngl_extmat[0:3,0:3]
    T=ngl_extmat[0:3,-1]
    campos=list(-(R.T)@T) 
    camdir=list((R.T)@np.array([0,0,1]))
    cam_dict={"campos":campos, "camdir":camdir}

# #     orientation=temp
# #     orientation = orientation.T
#     orientation = orientation.flatten()
#     orientation[-1] = 1
#     orientation[12]=0.
#     orientation[13]=0.
#     orientation[14]=0.
#     orientation *=1/100.
#     orientation[3] =0.
#     orientation[7] =0.
#     orientation[11]=0.

#     orientation -= np.array([24.451005287902106, 0.0, 0.0, -3.5999999046325684, 0.0, 24.451005287902106, 0.0, -3.5999999046325684, 0.0, 0.0, 24.451005287902106, -3.5999999046325684, 0.0, 0.0, 0.0, 0.0])
#     orientation += np.array([1,0,0,0,0,1,0,0,0,0,1,0,0,0,10.471975511965978,0])
    #     orientation=[0.7500277180472348,5.551115123125783e-17,0.6614063971272561,0,0.5078318190417079,0.6406823725596474,-0.5758758035029177,0,-0.4237514197376189,0.7678060285588679,0.48052953788400066,0,-4.437514490653162,8.040445929008314,5.0320935534975835,1]
#     orientation = [0.7683450340620741,-5.551115123125783e-17,0.6400358651139405,0,0.5491434688219112,0.5136687465056196,-0.659231271644166,0,-0.32876642055171756,0.8579885890053234,0.39467483053048313,0,-3.964409288439841,8.830108018321802,3.6037858154548483,1]
   # orientation = [0.9709872600909863,0,0.2391312207575565,0,0.23913122075743692,0.0000010000000000287557,-0.9709872600905007,0,-2.3913122074170623e-7,0.9999999999995,9.709872600405234e-7,0,-0.000002504176287919244,10.471975511960737,0.000010168154810102051,1]
    w.orientation=cam_dict
#     w.orientation = list(orientation)
    # with output:
    #     #print("set orientation ={}".format(w.orientation))
    #     ngl_dict=m.__dict__['_trait_values']
        
#         print("w = {}\n".format(w.__dict__))
#         print("init_world_matrix= {}".format(init_worldmatrix))

#         print("init_ngl_matrix= {}".format(init_ngl_matrix))
        
#         print("np.array(ngl_dict[\"_camera_orientation\"]).reshape(4,4).T = {}\n".format(np.array(ngl_dict["_camera_orientation"]).reshape(4,4).T))
#         print("camdict={}".format(cam_dict))
#         print("(np.linalg.inv(init_ngl_matrix)@np.array(ngl_dict[\"_camera_orientation\"]).reshape(4,4)).T{}".format(np.linalg.inv(init_ngl_matrix)@np.array(ngl_dict["_camera_orientation"]).reshape(4,4)))
        # set the bz visualizer camera orientation equal to this



output = Output(layout=Layout(width="400px"))
a_slider=widgets.FloatSlider(
    value=3.5,
    min=0,
    max=10.0,
    step=0.1,
    description='a:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

b_slider=widgets.FloatSlider(
    value=3.5,
    min=0,
    max=10.0,
    step=0.1,
    description='b:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

c_slider=widgets.FloatSlider(
    value=3.5,
    min=0,
    max=10.0,
    step=0.1,
    description='c:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

alpha_slider=widgets.FloatSlider(
    value=90.,
    min=0,
    max=180,
    step=0.1,
    description=r'$\alpha$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

beta_slider=widgets.FloatSlider(
    value=90.,
    min=0,
    max=180,
    step=0.1,
    description=r'$\beta$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

gamma_slider=widgets.FloatSlider(
    value=90.,
    min=0,
    max=180,
    step=0.1,
    description=r'$\gamma$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)


def fix_sliders_ab(a):
    # fix value of b to that of a
    return a

def fix_sliders_ac(a):
    # fix value of b and c sliders to that of a
    return a

def fix_sliders_alpha_beta(alpha):
    # fix value of b and c sliders to that of a
    return alpha

def fix_sliders_alpha_gamma(alpha):
    # fix value of b and c sliders to that of a
    return alpha

# set up link between a and b sliders
l_ab = widgets.dlink((a_slider, 'value'), (b_slider, 'value'),transform=fix_sliders_ab)
l_ab.link()
l_ab_flag=True # flags for keeping track of link status

# set up link between a and c sliders
l_ac = widgets.dlink((a_slider, 'value'), (c_slider, 'value'),transform=fix_sliders_ac)
l_ac.link()
l_ac_flag=True

# set up link between alpha and beta sliders
l_alpha_beta = widgets.dlink((alpha_slider, 'value'), (beta_slider, 'value'),transform=fix_sliders_alpha_beta)
l_alpha_beta.link()
l_alpha_beta_flag=True

# set up link between alpha and gamma sliders
l_alpha_gamma = widgets.dlink((alpha_slider, 'value'), (gamma_slider, 'value'),transform=fix_sliders_alpha_gamma)
l_alpha_gamma.link()
l_alpha_gamma_flag=True


# default settings for cubic crystal type
alpha_slider.value=90
beta_slider.value=90
gamma_slider.value=90
alpha_slider.disabled=True
beta_slider.disabled=True
gamma_slider.disabled=True
b_slider.disabled=True
c_slider.disabled=True

revert_lattice_params_button = Button(description="Revert lattice parameters.")
revert_lattice_params_button.on_click(on_revert_lattice_params)

save_lattice_params_button = Button(description="Save lattice parameters.")
save_lattice_params_button.on_click(on_save_lattice_params)

col2 = VBox([Text(value='Lattice parameters'),VBox([a_slider,b_slider,c_slider]), VBox([alpha_slider,beta_slider,gamma_slider, revert_lattice_params_button, save_lattice_params_button])])

# col2.set_title(0, 'Lattice vectors')
# col2.set_title(1, 'Axial angles')
# col2.set_title(2, 'List of atoms')




# Function to initialize crystal structure set up lattice vector and axial angle selection according to choice 
# of crystal family + centering
def reset_sliders(change):
    # crystal_family = str(tab.children[0].value)
    
    
   # get also the value of the centering chosen by the user before updating the crystal structure     
        
    
        global crystal_family_radio,alpha_slider, l_ab_flag, l_ac_flag, l_alpha_beta_flag, l_alpha_gamma_flag

        
        # If we want to have the lattice parameters automatically saved for future reversion
        # then we will need some logic to prevent overriding them when we change crystal
        # structure or centering
       
    # if type(change['owner']) == widgets.widget_selection.RadioButtons:
        #     save_lat_params=False
                # print("reset sliders: type(change[owner]) = {}".format(type(change['owner'])))
        # if change['owner'] = 
        crystal_family_radio.unobserve(reset_sliders, names='value')

        crystal_family = crystal_family_radio.value
        # with output:
        #     print("crystal family = {}".format(crystal_family))
        
        a_slider.unobserve(compute_BZ,names='value')
        b_slider.unobserve(compute_BZ,names='value')
        c_slider.unobserve(compute_BZ,names='value')
        alpha_slider.unobserve(compute_BZ,names='value')
        beta_slider.unobserve(compute_BZ,names='value')
        gamma_slider.unobserve(compute_BZ,names='value')

        # a_slider.unobserve(None) # temporarily unobserve callback to prevent looping
        # b_slider.unobserve(None)
        # c_slider.unobserve(None)
        # alpha_slider.unobserve(None)
        # beta_slider.unobserve(None)
        # gamma_slider.unobserve(None)
        
        if(crystal_family=='Cubic'):
              
            # lock b and c sliders to that of a
            # lock angles to 90,90,90
            centering_radio.options=['Primitive','Body-centered', 'Face-centered']
            alpha_slider.value=90
            beta_slider.value=90
            gamma_slider.value=90
            alpha_slider.disabled=True
            beta_slider.disabled=True
            gamma_slider.disabled=True
            b_slider.disabled=True
            c_slider.disabled=True

            if l_ab_flag==False:
                l_ab.link()
                l_ab_flag=True
            if l_ac_flag==False:
                l_ac.link()
                l_ac_flag=True
                
            if l_alpha_beta_flag==True:
                l_alpha_beta.unlink()
                l_alpha_beta_flag = False
                
            if l_alpha_gamma_flag==True:
                l_alpha_gamma.unlink()
                l_alpha_gamma_flag = False
                        

        elif(crystal_family=='Orthorhombic'):
           
            # vector lengths are all independent
            # lock angles to 90,90,90
            centering_radio.options=['Primitive','Base-centered','Body-centered', 'Face-centered']

            alpha_slider.value=90
            beta_slider.value=90
            gamma_slider.value=90
            alpha_slider.disabled=True
            beta_slider.disabled=True
            gamma_slider.disabled=True
            
            if l_ab_flag==True:
                l_ab.unlink()
                l_ab_flag=False

            if l_ac_flag==True:
                l_ac.unlink()
                l_ac_flag=False
                
            if l_alpha_beta_flag==True:
                l_alpha_beta.unlink()
                l_alpha_beta_flag=False
                
            if l_alpha_gamma_flag==True:
                l_alpha_gamma.unlink()
                l_alpha_gamma_flag=False
                
           
            b_slider.disabled=False
            c_slider.disabled=False
        

        elif(crystal_family=='Tetragonal'):
            
            centering_radio.options=['Primitive','Body-centered']
            alpha_slider.value=90
            beta_slider.value=90
            gamma_slider.value=90
            a_slider.value = 3.
            c_slider.value = 5.
            alpha_slider.disabled=True
            beta_slider.disabled=True
            gamma_slider.disabled=True
            
            
            # lock b slider value to that of a

            if l_ab_flag==False:
                l_ab.link()
                l_ab_flag=True

            if l_ac_flag==True:
                l_ac.unlink()
                l_ac_flag=False
                
            b_slider.disabled=True
            c_slider.disabled=False
            
            if l_alpha_beta_flag==False:
                l_alpha_beta.link()
                l_alpha_beta_flag=False
            
            if l_alpha_gamma_flag==False:
                l_alpha_gamma.link()
                l_alpha_gamma_flag=False
                                     


        elif(crystal_family=='Monoclinic'):
            # two lattice vectors meet at 90 degrees.
            # The third meets at an angle beta
            # all 3 lattice vectors can take unique values
            
            centering_radio.options=['Primitive','Base-centered']
            alpha_slider.value=90
            gamma_slider.value=90
            beta_slider.value = 60
            alpha_slider.disabled=True
            beta_slider.disabled=False
            gamma_slider.disabled=True
            a_slider.value = 3
            b_slider.value = 4
            c_slider.value = 5
           
            b_slider.disabled=False
            c_slider.disabled=False
           
            
            if l_ab_flag==True:
                l_ab.unlink()
                l_ab_flag=False

            if l_ac_flag==True:
                l_ac.unlink()
                l_ac_flag=False
                
            if l_alpha_beta_flag==True:
                l_alpha_beta.unlink()
                l_alpha_beta_flag=False
                
            if l_alpha_gamma_flag==True:
                l_alpha_gamma.unlink()
                l_alpha_gamma_flag=False
            
            

            
        elif(crystal_family=='Hexagonal'):
            # a=b with 120 degrees between
            # a and c are perpendicular. Similarly for b and c
            # all 3 lattice vectors can take unique values
          
            centering_radio.options=['Primitive']
            alpha_slider.value=90
            beta_slider.value=90
            a_slider.value = 3
            c_slider.value = 5
            gamma_slider.value=120
            alpha_slider.disabled=True
            gamma_slider.disabled=True
            
            b_slider.disabled=True
            c_slider.disabled=False
            
            if l_ab_flag==False:
                l_ab.link()
                l_ab_flag=True

            if l_ac_flag==True:
                l_ac.unlink()
                l_ac_flag=False
                
            if l_alpha_beta_flag==True:
                l_alpha_beta.unlink()
                l_alpha_beta_flag=False
                
            if l_alpha_gamma_flag==True:
                l_alpha_gamma.unlink()
                l_alpha_gamma_flag=False
            
            


        elif(crystal_family=='Rhombohedral'):
            # a=b=c. Only need to specify a and alpha.
            centering_radio.options=['Primitive']
            alpha_slider.disabled=False
            alpha_slider.value=60
            beta_slider.disabled=True
            gamma_slider.disabled=True
           
            b_slider.disabled=True
            c_slider.disabled=True
            
            if l_ab_flag==False:
                l_ab.link()
                l_ab_flag=True

            if l_ac_flag==False:
                l_ac.link()
                l_ac_flag=True
                
            if l_alpha_beta_flag==False:
                l_alpha_beta.link()
                l_alpha_beta_flag=True
                
            if l_alpha_gamma_flag==False:
                l_alpha_gamma.link()
                l_alpha_gamma_flag=True
            
            
            

        elif(crystal_family=='Triclinic'):
            # all 3 lattice vectors and angles may vary independently
        
            centering_radio.options=['Primitive']
           
            alpha_slider.value=30
            beta_slider.value=50
            gamma_slider.value=70
            
            
            if l_ab_flag==True:
                l_ab.unlink()
                l_ab_flag=False

        
            if l_ac_flag==True:
                l_ac.unlink()
                l_ac_flag=False
            
            if l_alpha_beta_flag==True:
                l_alpha_beta.unlink()
                l_alpha_beta_flag=False
            
            if l_alpha_gamma_flag==True:
                l_alpha_gamma.unlink()
                l_alpha_gamma_flag=False
            
            

            b_slider.value=3
            c_slider.value=4
            
            alpha_slider.disabled=False
            
            b_slider.disabled=False
            c_slider.disabled=False
            beta_slider.disabled=False
            gamma_slider.disabled=False
            
           
                 
        crystal_family_radio.observe(reset_sliders, names='value')


        a_slider.observe(compute_BZ,names='value')
        b_slider.observe(compute_BZ,names='value')
        c_slider.observe(compute_BZ,names='value')
        alpha_slider.observe(compute_BZ,names='value')
        beta_slider.observe(compute_BZ,names='value')
        gamma_slider.observe(compute_BZ,names='value')
        
        compute_BZ(change)
            
crystal_family_radio.observe(reset_sliders,names='value')
centering_radio.observe(reset_sliders,names='value')


def compute_BZ(c):
    global m,c1, crystal_family_radio,w, real_lattice
    
    crystal_family = crystal_family_radio.value
    # with output:
    #     print("compute_bz. crystal fam = {}, c == {}\n".format(crystal_family,c))

        
    try:
        a_slider.unobserve(compute_BZ,names='value')
    except:
        with output:
            print("error")
    b_slider.unobserve(compute_BZ,names='value')
    c_slider.unobserve(compute_BZ,names='value')
    alpha_slider.unobserve(compute_BZ,names='value')
    beta_slider.unobserve(compute_BZ,names='value')
    gamma_slider.unobserve(compute_BZ,names='value')
    
    
    
    # only update BZ if c['disabled'] != True
    # This prevents double updates due to linked sliders
    
    # if c['disabled'] == True:
    #     with output:
    #         print("returning == {}\n".format(c))
    #     return

    # get mag of lattice vec from slider and construct vector
    a = a_slider.value
    b = b_slider.value
    c = c_slider.value

    alpha=alpha_slider.value
    beta=beta_slider.value
    gamma=gamma_slider.value
    

    centering=centering_radio.value
    
    # save lattice parameters for retrieval after a change of crystal family
    
  
    

    mpos = [];
    msym = [];
        

    if(crystal_family=='Cubic'):
        if(centering=='Primitive'):

            real_lattice  =  np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) *a / 2.0
            mat = SimpleOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})

        elif(centering=='Body-centered'):
            
            real_lattice = np.array([[-1, 1, 1], [1, -1, 1], [1, 1, -1]]) *a / 2.0;
            mat = BodyCenteredOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})

        elif(centering=='Face-centered'):
            real_lattice=np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]]) *a/ 2.0;
            mat = FaceCenteredOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})                         

    if(crystal_family=='Orthorhombic'):
        if(centering=='Primitive'):
            real_lattice  =  np.array([[a, 0, 0], [0, b, 0], [0, 0, c]])
            mat = SimpleOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})

        elif(centering=='Base-centered'):

            real_lattice  =  np.array([[a, -b, 0], [a, b, 0], [0, 0, 2*c]]) / 2.0
            #real_lattice=np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]]) *a/ 2.0;
            mat = BaseCenteredOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})
        elif(centering=='Body-centered'):
            real_lattice  =  np.array([[-a, b, c], [a, -b, c], [a, b, -c]])/2
            mat = BodyCenteredOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})
        elif(centering=='Face-centered'):
            real_lattice  =  np.array([[0, b, c], [a, 0, c], [a, b, 0]])/2
            mat = FaceCenteredOrthorhombic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})
            
    
    if(crystal_family=='Triclinic'):
           
            c_x = c*np.cos(np.pi*(beta/180))
            c_y = (1./np.sin(np.pi*(90/180)))*c*(np.cos(np.pi*(alpha/180))-np.cos(np.pi*(beta/180))*np.cos(np.pi*(gamma/180)))
            c_z = np.sqrt(c**2-c_x**2-c_y**2)
            real_lattice  =  np.array([[a, 0, 0], [b*np.cos(np.pi*(gamma/180)), b*np.sin(np.pi*(gamma/180)), 0], [c_x, c_y, c_z]])               
            mat = Triclinic( size=(3,3,3), symbol='Cu', 
                              latticeconstant={'a': a, 'b': b, 'c': c,'alpha':alpha,'beta':beta,'gamma':gamma})

    if(crystal_family=='Monoclinic'):
        if(centering=='Primitive'):
            real_lattice=np.array([[a,0,0], [0, b, 0], [c*np.cos(np.pi*(beta/180)),0,c*np.sin(np.pi*(beta/180))]])
            mat = SimpleMonoclinic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c,'alpha':beta})
        elif(centering=='Base-centered'):
            real_lattice=np.array([[a/2,-b/2,0], [a/2, b/2, 0], [c*np.cos(np.pi*(beta/180)),0,c*np.sin(np.pi*(beta/180))]])
            mat = BaseCenteredMonoclinic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c,'alpha':beta})
    if(crystal_family=='Tetragonal'):
        if(centering=='Primitive'):
            real_lattice=np.array([[a,0,0], [0, a, 0], [0,0,c]])
            mat = SimpleTetragonal(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})
        elif(centering=='Body-centered'):
            real_lattice=np.array([[-a,a,c], [a, -a, c], [a,a,-c]])/2
            mat = BodyCenteredTetragonal(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          size=(3,3,3), symbol='Cu', 
                          latticeconstant={'a': a, 'b': b, 'c': c})
    if(crystal_family=='Rhombohedral'):
            mat = Triclinic( size=(3,3,3), symbol='Cu', 
                              latticeconstant={'a': a, 'b': a, 'c': a,'alpha':alpha,'beta':alpha,'gamma':alpha})

            # mat = bulk('Cu', 'rhombohedral', a=a,alpha=alpha,basis=[[0,0,0],[0,0,0],[0,0,0]])
            c_z=np.sqrt((1/3.)*(4*np.cos(np.pi*(alpha/2/180))**2-1))
            # rhombohedral/trigonal lattices aset not implemented in ase's lattice module
            real_lattice= a*np.array([[np.sin(np.pi*(alpha/2/180)), -1/np.sqrt(3)*np.sin(np.pi*(alpha/(2*180))), c_z], [0,2/np.sqrt(3)*np.sin(np.pi*(alpha/(2*180))), c_z],[-np.sin(np.pi*(alpha/2/180)), -1/(np.sqrt(3))*np.sin(np.pi*(alpha/2/180)), c_z]]) 

    if(crystal_family=='Hexagonal'):
        mat = Hexagonal(size=(3,3,3), symbol='Cu',latticeconstant={'a':a,'c':c})
        real_lattice= np.array([[a, 0, 0], [a/2, np.sqrt(3)/2*a, c],[0, 0, c],]) 


    w.cell = real_lattice.tolist()

    m.remove_component(c1)
    m.clear()

    c1 = m.add_component(nv.ASEStructure(mat))
    m.clear()

    m.add_spacefill(radius=0.4)
    m.add_unitcell()
    
    
    a_slider.observe(compute_BZ,names='value')
    b_slider.observe(compute_BZ,names='value')
    c_slider.observe(compute_BZ,names='value')
    alpha_slider.observe(compute_BZ,names='value')
    beta_slider.observe(compute_BZ,names='value')
    gamma_slider.observe(compute_BZ,names='value')


    
a_slider.observe(compute_BZ,names='value')
b_slider.observe(compute_BZ,names='value')
c_slider.observe(compute_BZ,names='value')
alpha_slider.observe(compute_BZ,names='value')
beta_slider.observe(compute_BZ,names='value')
gamma_slider.observe(compute_BZ,names='value')


# bt_compute.on_click(compute_BZ);

label1 = HTML(value = f"<div style='width: 400px; text-align:center;'><b><font color='blue'><font size=5>Structure</b></div>")
label2 = HTML(value = f"<div style='width: 400px; text-align:center;'><b><font color='blue'><font size=5>Brillouin zone</b></div>")

# orientation_sync_button = Button(description="Sync orientations")
# orientation_sync_button.on_click(on_orientation_sync)
# display(HBox([label1, label2]), HBox([m, VBox([w,orientation_sync_button])]), HBox([col0, col1,col2]))
display(HBox([label1, label2]), HBox([m, VBox([w])]), HBox([col0, col1,col2]))

# display(orientation_sync_button)
# try the following [0.7500277180472348,5.551115123125783e-17,0.6614063971272561,0,0.5078318190417079,0.6406823725596474,-0.5758758035029177,0,-0.4237514197376189,0.7678060285588679,0.48052953788400066,0,-4.437514490653162,8.040445929008314,5.0320935534975835,1]

In [ ]:
#[1,0,0,0,0,1,0,0,0,0,1,0,0,0,10.471975511965978,1] is init world vec
# [24.451005287902106, 0.0, 0.0, -3.5999999046325684, 0.0, 24.451005287902106, 0.0, -3.5999999046325684, 0.0, 0.0, 24.451005287902106, -3.5999999046325684, 0.0, 0.0, 0.0, 1.0
# is init orientation
# np.array([1,0,0,0,0,1,0,0,0,0,1,0,0,0,10.471975511965978,1]).reshape((4,4)).T

In [ ]:
# a = np.array([[24.45100529 , 0.    ,      0.    ,     -3.5999999 ],
#  [ 0.   ,      24.45100529,  0.   ,      -3.5999999 ],
#  [ 0.   ,       0.       ,  24.45100529 ,-3.5999999 ],
#  [ 0.  ,        0.      ,    0.       ,   1.        ]])
# np.linalg.inv(a)@a

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Legend**
(How to use the interactive visualization)

### Interactive figures

There are two visualizers for the real space lattice structure (top left) and its 1st Brillouin zone in reciprocal space (top right) respectively. One can left-click
and rotate both structures. Middle wheel of the mouse can zoom in and zoom out
the structures. In the 1st Brillouin zone visualizer, the high symmetry points
are also shown in the figure.

### Controls

There are a collection of controls under the two visualizers. Firstly, one can use the "Crystal family" dropdown menu to choose
from the possible crystal families in 3D, such as cubic, orthorhombic, tetragonal, 
monoclinic, etc. After having selected the crystal family one is interested in studying, the user may then select the type of centering they wish to investigate (primitive, face-centered, body-centered, base-centered). Of course, certain crystal families only possess a subset of these possible types of centering and therefore certain options shall be greyed out depending on the choice for the crystal family.

Finally, users can change the lattice parameters (lattice vector magnitudes and axial angles) on the bottom right panel. Recall that certain selections of the crystal family shall impose restrictions on the allowable lattice parameters and therefore some of the sliders shall be locked accordingly.